注意，PeftModel只支持相同adapter的模型，而PeftMixedModel支持不同adapter的模型。

PeftModel一次只能激活一个adapter，而PeftMixedModel可以激活多个adapter。

In [1]:
from peft.tuners import MixedModel

base_model = 'Qwen/Qwen2-7B-Instruct'  # load the base model, e.g. from transformers

In [2]:
# load base model
from transformers import AutoModel
model = AutoModel.from_pretrained(base_model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
# load the model with PEFT
from peft import PeftModel, LoraConfig

rank = 4
LoRA_amount = 3

peft_config = LoraConfig(
    inference_mode=False,
    r=rank,
    lora_alpha=32,
    lora_dropout=0.1
)
model = PeftModel(model, peft_config, adapter_name="0")
for LoRA_index in range(1, LoRA_amount):
    model.add_adapter(str(LoRA_index), peft_config)

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/opt/miniconda3/envs/memo/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [4]:
model

PeftModel(
  (base_model): LoraModel(
    (model): Qwen2Model(
      (embed_tokens): Embedding(152064, 3584)
      (layers): ModuleList(
        (0-27): 28 x Qwen2DecoderLayer(
          (self_attn): Qwen2SdpaAttention(
            (q_proj): lora.Linear(
              (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
              (lora_dropout): ModuleDict(
                (0): Dropout(p=0.1, inplace=False)
                (1): Dropout(p=0.1, inplace=False)
                (2): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (0): Linear(in_features=3584, out_features=4, bias=False)
                (1): Linear(in_features=3584, out_features=4, bias=False)
                (2): Linear(in_features=3584, out_features=4, bias=False)
              )
              (lora_B): ModuleDict(
                (0): Linear(in_features=4, out_features=3584, bias=False)
                (1): Linear(in_features=4, out_features=3584, bias=

In [5]:
model.set_adapter('2')

In [6]:
model.active_adapters

['2']

In [7]:
# 这会导致报错
model.set_adapter(['1', '2'])

TypeError: unhashable type: 'list'